In [62]:
import pandas as pd

data = [[10, '2019-12-10'], 
        [8, '2020-1-13'], 
        [5, '2020-2-16'], 
        [7, '2020-3-8'], 
        [4, '2020-5-17'], 
        [1, '2020-10-24'], 
        [6, '2021-1-5']]
drivers = pd.DataFrame(data, 
                       columns=['driver_id', 
                                'join_date']).astype({'driver_id':'Int64', 
                                                      'join_date':'datetime64[ns]'})
data = [[6, 75, '2019-12-9'],
        [1, 54, '2020-2-9'], 
        [10, 63, '2020-3-4'], 
        [19, 39, '2020-4-6'], 
        [3, 41, '2020-6-3'], 
        [13, 52, '2020-6-22'], 
        [7, 69, '2020-7-16'], 
        [17, 70, '2020-8-25'], 
        [20, 81, '2020-11-2'], 
        [5, 57, '2020-11-9'], 
        [2, 42, '2020-12-9'], 
        [11, 68, '2021-1-11'], 
        [15, 32, '2021-1-17'], 
        [12, 11, '2021-1-19'], 
        [14, 18, '2021-1-27']]
rides = pd.DataFrame(data, 
                     columns=['ride_id', 
                              'user_id', 
                              'requested_at']).astype({'ride_id':'Int64', 
                                                       'user_id':'Int64', 
                                                       'requested_at':'datetime64[ns]'})
data = [[10, 10, 63, 38], 
        [13, 10, 73, 96], 
        [7, 8, 100, 28], 
        [17, 7, 119, 68], 
        [20, 1, 121, 92], 
        [5, 7, 42, 101], 
        [2, 4, 6, 38], 
        [11, 8, 37, 43], 
        [15, 8, 108, 82], 
        [12, 8, 38, 34], 
        [14, 1, 90, 74]]
accepted_rides = pd.DataFrame(data, 
                              columns=['ride_id', 
                                       'driver_id', 
                                       'ride_distance', 
                                       'ride_duration']).astype({'ride_id':'Int64', 
                                                                 'driver_id':'Int64', 
                                                                 'ride_distance':'Int64', 
                                                                 'ride_duration':'Int64'})
display(drivers, rides, accepted_rides)

,driver_id,join_date
0,10,2019-12-10
1,8,2020-01-13
2,5,2020-02-16
3,7,2020-03-08
4,4,2020-05-17
5,1,2020-10-24
6,6,2021-01-05


,ride_id,user_id,requested_at
0,6,75,2019-12-09
1,1,54,2020-02-09
2,10,63,2020-03-04
3,19,39,2020-04-06
4,3,41,2020-06-03
5,13,52,2020-06-22
6,7,69,2020-07-16
7,17,70,2020-08-25
8,20,81,2020-11-02
9,5,57,2020-11-09


,ride_id,driver_id,ride_distance,ride_duration
0,10,10,63,38
1,13,10,73,96
2,7,8,100,28
3,17,7,119,68
4,20,1,121,92
5,5,7,42,101
6,2,4,6,38
7,11,8,37,43
8,15,8,108,82
9,12,8,38,34


In [63]:
drivers = drivers.sort_values(by=["join_date"])
drivers["year"] = drivers["join_date"].dt.year
drivers["month"] = drivers["join_date"].dt.month
drivers = drivers[drivers["year"]<=2020]

active_drivers = drivers.groupby(["year", "month"])[["driver_id"]].count().reset_index()
active_drivers["active_drivers"] = active_drivers["driver_id"].cumsum()
active_drivers = active_drivers[active_drivers["year"]==2020]

month_df = pd.DataFrame()
month_df["month"] = range(1, 13)
month_df = month_df.merge(active_drivers[["month", "active_drivers"]], how="left")
month_df["active_drivers"] = month_df["active_drivers"].ffill()

rides = rides.sort_values(by=["requested_at"])
rides["year"] = rides["requested_at"].dt.year
rides["month"] = rides["requested_at"].dt.month
rides = rides[rides["year"]==2020]

rides = rides.merge(accepted_rides[["ride_id"]], how="inner")
rides = rides.groupby(["month"])[["ride_id"]].count()
rides = rides.reset_index().rename(columns={"ride_id": "accepted_rides"})

month_df = month_df.merge(rides[["month", "accepted_rides"]], how="left")
month_df["accepted_rides"] = month_df["accepted_rides"].fillna(0)

month_df

,month,active_drivers,accepted_rides
0,1,2,0
1,2,3,0
2,3,4,1
3,4,4,0
4,5,5,0
5,6,5,1
6,7,5,1
7,8,5,1
8,9,5,0
9,10,6,0


References: [1] https://leetcode.com/problems/hopper-company-queries-i/?lang=pythondata